In [1]:
import abc
from collections import *
import random
from typing import *

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
"""
Tic Tac Toe implementation
"""

EMPTY = 0
PLAYER = 1
OPPONENT = -1

Move = Tuple[int, int]
PlayerId = int
Reward = float

COORDINATES = [
    (1, 1),                             # Middle first
    (0, 0), (2, 0), (0, 2), (2, 2),     # Corners
    (1, 0), (0, 1), (1, 2), (2, 1)      # The rest
]

COMBINATIONS = [
    # diagonals
    {(0, 0), (1, 1), (2, 2)},
    {(2, 0), (1, 1), (0, 2)},
    # rows
    {(0, 0), (0, 1), (0, 2)},
    {(1, 0), (1, 1), (1, 2)},
    {(2, 0), (2, 1), (2, 2)},
    # cols
    {(0, 0), (1, 0), (2, 0)},
    {(0, 1), (1, 1), (2, 1)},
    {(0, 2), (1, 2), (2, 2)}
]

def next_player(player: PlayerId):
    if player == PLAYER:
        return OPPONENT
    return PLAYER

class TicTacToe:
    def __init__(self, grid):
        self.grid = grid
    
    @classmethod
    def empty(cls):
        return cls(grid=np.zeros(shape=(3, 3)))
    
    def clone(self):
        return TicTacToe(grid=self.grid.copy())
    
    def __eq__(self, other):
        return np.array_equal(self.grid, other.grid)
    
    def __hash__(self):
        return hash(self.grid.tostring())

    def available_moves(self):
        moves = []
        for move in COORDINATES:
            if self.grid[move] == EMPTY:
                moves.append(move)
        return moves
    
    def play(self, player: PlayerId, move: Move) -> Reward:
        if self.grid[move] != EMPTY:
            raise Exception("Invalid move: " + str(move))
        
        self.grid[move] = player
        winner = self.winner()
        if winner == player:
            return 1
        elif winner == next_player(player):
            return -1
        return 0
    
    def is_over(self) -> bool:
        return self.filled() or self.winner() != EMPTY
    
    def winner(self):
        for combi in COMBINATIONS:
            player_count = 0
            opponent_count = 0
            for position in combi:
                if self.grid[position] == PLAYER:
                    player_count += 1
                elif self.grid[position] == OPPONENT:
                    opponent_count += 1
            if player_count == 3:
                return PLAYER
            elif opponent_count == 3:
                return OPPONENT
        return EMPTY
    
    def filled(self):
        for pos in COORDINATES:
            if self.grid[pos] == EMPTY:
                return False
        return True

In [3]:
"""
A famous way to win a game
x x o
    x
o   o
"""

game = TicTacToe.empty()
print(game.play(PLAYER, (0, 0)))
print(game.play(OPPONENT, (2, 2)))
print(game.play(PLAYER, (0, 1)))
print(game.play(OPPONENT, (0, 2)))
print(game.play(PLAYER, (1, 2)))
print(game.play(OPPONENT, (2, 0)))
print(game.grid)
print(game.play(PLAYER, (1, 1)))
print(game.play(OPPONENT, (2, 1)))
print(game.grid)
print("winner:", game.winner())

assert TicTacToe.empty() == TicTacToe.empty()
assert hash(TicTacToe.empty()) == hash(TicTacToe.empty())

0
0
0
0
0
0
[[ 1.  1. -1.]
 [ 0.  0.  1.]
 [-1.  0. -1.]]
0
1
[[ 1.  1. -1.]
 [ 0.  1.  1.]
 [-1. -1. -1.]]
winner: -1


In [14]:
"""
An environment for an agent to play against
"""


class Agent(abc.ABC):    
    @abc.abstractmethod
    def get_action(self, game: TicTacToe) -> Move:
        pass


class TicTacToeEnv:
    def __init__(self, opponent: Agent):
        self.game: TicTacToe = TicTacToe.empty()
        self.opponent: Agent = opponent
    
    def reset(self):
        self.game = TicTacToe.empty()
        if random.choice([PLAYER, OPPONENT]) == OPPONENT:
            move = self.opponent.get_action(self.game)
            self.game.play(OPPONENT, move)
        return self.state
        
    @property
    def state(self):
        return self.game.grid
    
    def is_over(self):
        return self.game.is_over()
    
    def step(self, move: Move) -> Reward:
        reward = self.game.play(PLAYER, move)
        if self.game.is_over():
            return reward
        
        move = self.opponent.get_action(self.game)
        reward = self.game.play(OPPONENT, move)
        return -1 * reward

In [18]:
"""
Trying out the environment with another random agent
"""


class RandomAgent(Agent):
    def __init__(self):
        pass
    
    def get_action(self, game: TicTacToe) -> Move:
        moves = game.available_moves()
        return random.choice(moves)


def play_episode(env, agent, with_log=False):
    total_reward = 0.
    env.reset()
    while not env.is_over():
        if with_log:
            print(env.game.grid)
        move = agent.get_action(env.game)
        reward = env.step(move)
        total_reward += reward
    if with_log:
        print(env.game.grid)
    return total_reward, env.state


env=TicTacToeEnv(opponent=RandomAgent())
result, state = play_episode(env, agent=RandomAgent())
print(result)
print(state)

# basics stats
stats = defaultdict(int)
for _ in range(1000):
    result, _ = play_episode(env, agent=RandomAgent())
    stats[result] += 1
print(stats)

-1.0
[[-1.  1. -1.]
 [-1.  0.  1.]
 [-1.  0.  1.]]
defaultdict(<class 'int'>, {-1.0: 452, 1.0: 433, 0.0: 115})


In [39]:
"""
All powerful agent: search throught he whole state space
"""


class AllKnowingAI(Agent):
    def __init__(self, player: PlayerId):
        self.player = player
        self.memo = {}
    
    def get_action(self, game: TicTacToe) -> Move:
        return self._best_action(game, self.player)[0]

    def _best_action(self, game: TicTacToe, player: PlayerId) -> Tuple[Move, Reward]:
        if (game, player) in self.memo:
            return self.memo[(game, player)]

        if game.is_over():
            return None, 0.

        best_move = None
        best_score = -2.0
        for move in game.available_moves():
            next_game = game.clone()
            score = next_game.play(player, move)                
            _, next_score = self._best_action(next_game, next_player(player))
            score -= 0.9 * next_score  # negate the score of opponent + discount it to prefer early victories
            if score > best_score:
                best_score = score
                best_move = move

        self.memo[(game, player)] = best_move, best_score
        return best_move, best_score
        

"""
Trying this agent
"""


env=TicTacToeEnv(opponent=RandomAgent())
result, state = play_episode(env, agent=AllKnowingAI(player=PLAYER), with_log=True)
print(result)

[[ 0.  0. -1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[[ 0.  0. -1.]
 [ 0.  1. -1.]
 [ 0.  0.  0.]]
[[ 0.  0. -1.]
 [ 0.  1. -1.]
 [ 0. -1.  1.]]
[[ 1.  0. -1.]
 [ 0.  1. -1.]
 [ 0. -1.  1.]]
1.0
